In [2]:
from google.colab import files
files.upload()  # اختر ملف kaggle.json من جهازك

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"saraahishamm","key":"38dc15404e0230c048a10167d8b6f73c"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 97% 336M/346M [00:02<00:00, 121MB/s]
100% 346M/346M [00:02<00:00, 149MB/s]


In [5]:
import zipfile
with zipfile.ZipFile("intel-image-classification.zip", 'r') as zip_ref:
    zip_ref.extractall("intel-image-classification")


In [6]:
import os
os.listdir("intel-image-classification")

['seg_train', 'seg_pred', 'seg_test']

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

In [8]:
#Defining size
batch_size = 32
img_height = 150
img_width = 150


Loading Data¶


In [9]:
train_folder="intel-image-classification/seg_train/seg_train"
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14034 files belonging to 6 classes.


In [10]:
valid_folder="intel-image-classification/seg_test/seg_test"
val_ds = tf.keras.utils.image_dataset_from_directory(
  valid_folder,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3000 files belonging to 6 classes.


In [11]:
#Labels
label_to_class_name = dict(zip(range(len(train_ds.class_names)), train_ds.class_names))
label_to_class_name

{0: 'buildings',
 1: 'forest',
 2: 'glacier',
 3: 'mountain',
 4: 'sea',
 5: 'street'}

In [12]:
#Size
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 150, 150, 3)
(32,)




Data preprocessing


In [13]:
train_ds=train_ds.map(lambda x,y:(x/255,y))
val_ds=val_ds.map(lambda x,y:(x/255,y))

In [14]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


#Training Models¶¶




#VGG16 Model


In [15]:
VGG_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(150,150,3),
                   pooling='max',classes=6,
                   weights='imagenet')
VGG_model.add(pretrained_model)
VGG_model.add(Flatten())
VGG_model.add(Dense(512, activation='relu'))
VGG_model.add(BatchNormalization())  # Batch Normalization layer
VGG_model.add(Dropout(0.5))
VGG_model.add(Dense(6, activation='softmax'))
pretrained_model.trainable=False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
VGG_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [17]:
VGG_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 512)                 │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,982,470 (57.15 MB)

 Trainable params: 266,758 (1.02 MB)

 Non-trainable params: 14,715,712 (56.14 MB)

In [18]:
epochs=15
history2 = VGG_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 64s 120ms/step - accuracy: 0.7466 - loss: 0.7238 - val_accuracy: 0.8493 - val_loss: 0.4277
Epoch 2/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 34s 78ms/step - accuracy: 0.8382 - loss: 0.4450 - val_accuracy: 0.8513 - val_loss: 0.3943
Epoch 3/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 39s 89ms/step - accuracy: 0.8481 - loss: 0.4076 - val_accuracy: 0.8513 - val_loss: 0.3989
Epoch 4/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 34s 78ms/step - accuracy: 0.8560 - loss: 0.3890 - val_accuracy: 0.8540 - val_loss: 0.4092
Epoch 5/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 45s 88ms/step - accuracy: 0.8593 - loss: 0.3792 - val_accuracy: 0.8433 - val_loss: 0.4169
Epoch 6/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.8652 - loss: 0.3611 - val_accuracy: 0.8523 - val_loss: 0.3985
Epoch 7/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 34s 78ms/step - accuracy: 0.8692 - loss: 0.3577 - val_accuracy: 0.8543 - val_loss: 0.3939
Epoch 8/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 79ms/step - accuracy: 0.8722 - loss: 0.3546 -

In [19]:
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.8900 - loss: 0.3018
Total Loss : 0.2975
Total Accuracy : 0.8917


In [20]:
ModelLoss, ModelAccuracy = VGG_model.evaluate(val_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.8433 - loss: 0.4218
Total Loss : 0.4154
Total Accuracy : 0.8517


In [22]:
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy  {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 28s 65ms/step - accuracy: 0.9096 - loss: 0.2484
Total Loss : 0.2456
Total Accuracy  0.9090


In [23]:
ModelLoss, ModelAccuracy = VGG_model.evaluate(val_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy  {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.8497 - loss: 0.4342
Total Loss : 0.4248
Total Accuracy  0.8540


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# الحصول على القيم الحقيقية والتوقعات من مجموعة التحقق
true_labels = []
predicted_labels = []

# تفريغ بيانات val_ds
for images, labels in val_ds:
    true_labels.extend(labels.numpy())  # القيم الحقيقية
    predictions = np.argmax(mobile_net.predict(images), axis=1)  # التوقعات
    predicted_labels.extend(predictions)

# تحويل القيم إلى مصفوفات
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# إنشاء مصفوفة الالتباس
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# عرض مصفوفة الالتباس
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=list(label_to_class_name.values()))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()




#MobileNet Model


In [24]:
mobile_net = Sequential()
pretrained_model= tf.keras.applications.MobileNetV2(include_top=False,
                   input_shape=(150,150,3),
                   pooling='max',classes=6,
                   weights='imagenet')
mobile_net.add(pretrained_model)
mobile_net.add(Flatten())
mobile_net.add(Dense(512, activation='relu'))
mobile_net.add(BatchNormalization())  # Batch Normalization layer
mobile_net.add(Dropout(0.5))
mobile_net.add(Dense(6, activation='softmax'))
pretrained_model.trainable=False

<ipython-input-24-4a2e8e1ff51e>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  pretrained_model= tf.keras.applications.MobileNetV2(include_top=False,


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [25]:
mobile_net.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [26]:
mobile_net.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 1280)                │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,918,982 (11.14 MB)

 Trainable params: 659,974 (2.52 MB)

 Non-trainable params: 2,259,008 (8.62 MB)

In [27]:
epochs=15
history3 = mobile_net.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 27s 39ms/step - accuracy: 0.8201 - loss: 0.5546 - val_accuracy: 0.8733 - val_loss: 0.3602
Epoch 2/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.8944 - loss: 0.3020 - val_accuracy: 0.8927 - val_loss: 0.2865
Epoch 3/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.8988 - loss: 0.2675 - val_accuracy: 0.8983 - val_loss: 0.2778
Epoch 4/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9106 - loss: 0.2471 - val_accuracy: 0.8833 - val_loss: 0.3033
Epoch 5/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9135 - loss: 0.2376 - val_accuracy: 0.8927 - val_loss: 0.2854
Epoch 6/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9149 - loss: 0.2271 - val_accuracy: 0.8947 - val_loss: 0.2917
Epoch 7/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9204 - loss: 0.2189 - val_accuracy: 0.8953 - val_loss: 0.2836
Epoch 8/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9215 - loss: 0.2121 - val_

In [28]:
# حساب الـ Test Loss والـ Test Accuracy
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

# طباعة النتائج
print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.9096 - loss: 0.2484
Total Loss : 0.2456
Total Accuracy : 0.9090


In [29]:
# حساب الـ Test Loss والـ Test Accuracy
ModelLoss, ModelAccuracy = VGG_model.evaluate( val_ds)

# طباعة النتائج
print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.8497 - loss: 0.4342
Total Loss : 0.4248
Total Accuracy : 0.8540


In [30]:
epochs=25
history3 = mobile_net.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9345 - loss: 0.1723 - val_accuracy: 0.8737 - val_loss: 0.3682
Epoch 2/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9341 - loss: 0.1757 - val_accuracy: 0.8877 - val_loss: 0.3512
Epoch 3/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9370 - loss: 0.1660 - val_accuracy: 0.8797 - val_loss: 0.3715
Epoch 4/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9356 - loss: 0.1632 - val_accuracy: 0.8773 - val_loss: 0.3892
Epoch 5/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9347 - loss: 0.1671 - val_accuracy: 0.8717 - val_loss: 0.3986
Epoch 6/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9369 - loss: 0.1658 - val_accuracy: 0.8727 - val_loss: 0.3994
Epoch 7/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9405 - loss: 0.1584 - val_accuracy: 0.8547 - val_loss: 0.4550
Epoch 8/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9422 - loss: 0.1533 - val_

In [31]:
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.9096 - loss: 0.2484
Total Loss : 0.2456
Total Accuracy : 0.9090


In [32]:
ModelLoss, ModelAccuracy = VGG_model.evaluate(val_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.8497 - loss: 0.4342
Total Loss : 0.4248
Total Accuracy : 0.8540
